***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@ntu.edu.tw </p>
### <p> Master Program in Statistics, National Taiwan University, Taipei, Taiwan.</p> 
---

# Input

In [1]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime


In [2]:
# Input
PATH1='../data/seedinds.txt'
PATH2='../data/bilabels.txt'
PATH3='../data/seedinds_neighborregions.txt'
PATH4='../data/ResNet18_PlantDisease_45K_Spec200.csv'
PATH5='../data/ResNet18_PlantDisease_45K_Values.csv'


#===  parameters ========================================================
TRIALS    = 1
timestr   = ''
REG_COLUMN = "Spec200"
RAW_2D_DATA = False

if RAW_2D_DATA: # 2D
    from CNN_Modules import ME_CNN
else: # 1D
    from CNN_Modules_1D import ME_CNN


# (1)PATH1
df1 = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df1.to_numpy().T[0]
NUM_region = len(region)

# (2)PATH2
df2 = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df2.to_numpy()

print('region:', region)
print('region amount:', len(region))


# (4)PATH4. Have to be here. The following neighboring process needs this information
df4 = pandas.read_csv(PATH4)
display(df4.head())
#all_region_index  = df.iloc[:,REGION_INDEX_LOC].to_numpy().astype(int)
all_region_index  = df4[REG_COLUMN].to_numpy().astype(int)
print("all_region_index\n",len(all_region_index), all_region_index)


# (4)PATH4 reset label
label=df4['Label'].value_counts(sort=False).keys()
_=[df4['Label'].replace(to_replace=label[i], value=i, inplace=True) for i in range(len(label))]
display(df4['Label'].value_counts(sort=False))
test_label_answer = df4["Label"].to_numpy()

region: [ 33  95 131  24 125  10  94 192 135  25 139 168 110  60 134  27  58  28]
region amount: 18


,X1,X2,X3,Class,Label,Spec200
0,-6.131136,-11.714952,14.559869,Cherry,2,100
1,4.653914,-15.913769,8.271565,Cherry,2,6
2,4.666949,-15.911179,8.316633,Cherry,2,6
3,-1.160013,-8.344077,19.694381,Cherry,2,166
4,-0.430728,-5.483346,17.380125,Cherry,2,170


all_region_index
 43217 [100   6   6 ... 186  83  32]


0    5000
1    4386
2    5000
3    5000
4    4457
5    4876
6    5000
7    4498
8    5000
Name: Label, dtype: int64

In [3]:
df1
import pandas as pd
print(pd.__version__)

1.1.3


In [4]:
# (5)PATH5     convert the embedded data into the pickle file
df5 = pandas.read_csv(PATH5)
test_array = df5.to_numpy()


#save
PATH5='../data/embedded_data.pickle'   #replace original PATH 5
with open(PATH5, 'wb') as f:
    pickle.dump([test_array, test_label_answer], f)
print(test_label_answer, "\n", test_array)

[1 1 1 ... 8 8 8] 
 [[ -6.131136   -11.71495238  14.55986935]
 [  4.65391405 -15.91376892   8.27156454]
 [  4.66694856 -15.91117949   8.31663274]
 ...
 [ -1.49778226  27.08358869 -12.30754204]
 [ -0.53695937  14.94667994  -7.44467661]
 [ -6.71398626   0.54063622   0.42243439]]


# Get neighbors

In [5]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

neighbor amount:  90
[[138  55  48  87   1]
 [ 71 158 144  81  69]
 [ 80  65  77  57 164]
 [ 15  63 146 115 109]
 [ 68 127  36  67 174]
 [197 120  96 130  50]
 [169 145 137 151  72]
 [116 147  91  64 198]
 [165  37  73 117 171]
 [ 17 156 175  88  66]
 [185 162  21 136  13]
 [113  20  84 199  16]
 [ 54 108 111  52   7]
 [ 61 160 121  26 146]
 [155 161  41  86 157]
 [178  56   5  42  75]
 [126 180  89 179  72]
 [ 59 153  44 167  18]]
neighbor amount:  88


[[138, 55, 48, 87, 1],
 [71, 158, 144, 81, 69],
 [80, 65, 77, 57, 164],
 [15, 63, 146, 115, 109],
 [68, 127, 36, 67, 174],
 [197, 120, 96, 130, 50],
 [169, 145, 137, 151, 72],
 [116, 147, 91, 64, 198],
 [165, 37, 73, 117, 171],
 [17, 156, 175, 88, 66],
 [185, 162, 21, 136, 13],
 [113, 20, 84, 199, 16],
 [54, 108, 111, 52, 7],
 [61, 160, 121, 26],
 [155, 161, 41, 86, 157],
 [178, 56, 5, 42, 75],
 [126, 180, 89, 179],
 [59, 153, 44, 167, 18]]

In [6]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

[[33, 138, 55, 48, 87, 1],
 [95, 71, 158, 144, 81, 69],
 [131, 80, 65, 77, 57, 164],
 [24, 15, 63, 146, 115, 109],
 [125, 68, 127, 36, 67, 174],
 [10, 197, 120, 96, 130, 50],
 [94, 169, 145, 137, 151, 72],
 [192, 116, 147, 91, 64, 198],
 [135, 165, 37, 73, 117, 171],
 [25, 17, 156, 175, 88, 66],
 [139, 185, 162, 21, 136, 13],
 [168, 113, 20, 84, 199, 16],
 [110, 54, 108, 111, 52, 7],
 [60, 61, 160, 121, 26],
 [134, 155, 161, 41, 86, 157],
 [27, 178, 56, 5, 42, 75],
 [58, 126, 180, 89, 179],
 [28, 59, 153, 44, 167, 18]]

In [7]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

In [8]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
# 20240319
if RAW_2D_DATA: # 2D
    print("")
else: # 1D
    test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

test_array: (43217, 3)
training size: 22653


# CNN original

In [9]:
# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # fill up training array
        
    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
        


    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

n, p1, p2 0 0 0


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)               

20387/20387 [==============================] - 1s 38us/step - loss: 1.4248 - accuracy: 0.3945 - val_loss: 1.4500 - val_accuracy: 0.3967
Epoch 8/80
20387/20387 [==============================] - 1s 38us/step - loss: 1.4203 - accuracy: 0.3944 - val_loss: 1.4505 - val_accuracy: 0.4082
Epoch 9/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.4109 - accuracy: 0.3960 - val_loss: 1.4373 - val_accuracy: 0.3866
Epoch 10/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.4079 - accuracy: 0.3972 - val_loss: 1.4378 - val_accuracy: 0.3866
Epoch 11/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.3992 - accuracy: 0.4047 - val_loss: 1.4478 - val_accuracy: 0.3883
Epoch 12/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.3998 - accuracy: 0.4016 - val_loss: 1.4345 - val_accuracy: 0.3844
Epoch 13/80
20387/20387 [==============================] - 1s 42us/step - loss: 1.3963 - accuracy: 0.4012 - val_loss: 1.4216 -

20387/20387 [==============================] - 1s 41us/step - loss: 1.3770 - accuracy: 0.4033 - val_loss: 1.4034 - val_accuracy: 0.3901
Epoch 24/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.3746 - accuracy: 0.4005 - val_loss: 1.3992 - val_accuracy: 0.3725
Epoch 25/80
20387/20387 [==============================] - 1s 38us/step - loss: 1.3680 - accuracy: 0.4087 - val_loss: 1.3973 - val_accuracy: 0.3782
Epoch 26/80
20387/20387 [==============================] - 1s 38us/step - loss: 1.3676 - accuracy: 0.4062 - val_loss: 1.3813 - val_accuracy: 0.3769
Epoch 27/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.3622 - accuracy: 0.4087 - val_loss: 1.3889 - val_accuracy: 0.3998
Epoch 28/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.3621 - accuracy: 0.4053 - val_loss: 1.3811 - val_accuracy: 0.3967
Epoch 29/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.3634 - accuracy: 0.4075 - val_loss: 1.3904

20387/20387 [==============================] - 1s 44us/step - loss: 1.2127 - accuracy: 0.5027 - val_loss: 1.1859 - val_accuracy: 0.5481
Epoch 40/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.1469 - accuracy: 0.5388 - val_loss: 1.1085 - val_accuracy: 0.5812
Epoch 41/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.0816 - accuracy: 0.5724 - val_loss: 1.0543 - val_accuracy: 0.5883
Epoch 42/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.0205 - accuracy: 0.5983 - val_loss: 1.0106 - val_accuracy: 0.6050
Epoch 43/80
20387/20387 [==============================] - 1s 38us/step - loss: 0.9769 - accuracy: 0.6118 - val_loss: 0.9521 - val_accuracy: 0.6447
Epoch 44/80
20387/20387 [==============================] - 1s 39us/step - loss: 0.9448 - accuracy: 0.6200 - val_loss: 0.9268 - val_accuracy: 0.6447
Epoch 45/80
20387/20387 [==============================] - 1s 38us/step - loss: 0.9144 - accuracy: 0.6353 - val_loss: 0.9120

Train on 20387 samples, validate on 2266 samples
Epoch 1/80
20387/20387 [==============================] - 1s 53us/step - loss: 2.4277 - accuracy: 0.2109 - val_loss: 1.8021 - val_accuracy: 0.3102
Epoch 2/80
20387/20387 [==============================] - 1s 42us/step - loss: 1.6251 - accuracy: 0.3558 - val_loss: 1.5520 - val_accuracy: 0.3447
Epoch 3/80
20387/20387 [==============================] - 1s 44us/step - loss: 1.4961 - accuracy: 0.3753 - val_loss: 1.5020 - val_accuracy: 0.3526
Epoch 4/80
20387/20387 [==============================] - 1s 40us/step - loss: 1.4684 - accuracy: 0.3720 - val_loss: 1.4838 - val_accuracy: 0.3680
Epoch 5/80
20387/20387 [==============================] - 1s 40us/step - loss: 1.4542 - accuracy: 0.3780 - val_loss: 1.4683 - val_accuracy: 0.3619
Epoch 6/80
20387/20387 [==============================] - 1s 44us/step - loss: 1.4370 - accuracy: 0.3835 - val_loss: 1.4510 - val_accuracy: 0.3756
Epoch 7/80
20387/20387 [==============================] - 1s 41us/ste

In [10]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

(5, 43217) (5, 43217, 18)


# CNN combination and removal

In [11]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

153

n, p1, p2 0 0 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 84

Epoch 7/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.4431 - accuracy: 0.3872 - val_loss: 1.4525 - val_accuracy: 0.3716
Epoch 8/80
20387/20387 [==============================] - 1s 38us/step - loss: 1.4353 - accuracy: 0.3869 - val_loss: 1.4242 - val_accuracy: 0.3672
Epoch 9/80
20387/20387 [==============================] - 1s 41us/step - loss: 1.4292 - accuracy: 0.3934 - val_loss: 1.4273 - val_accuracy: 0.3888
Epoch 10/80
20387/20387 [==============================] - 1s 38us/step - loss: 1.4182 - accuracy: 0.3942 - val_loss: 1.4187 - val_accuracy: 0.4086
Epoch 11/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.4129 - accuracy: 0.3961 - val_loss: 1.4226 - val_accuracy: 0.3835
Epoch 12/80
20387/20387 [==============================] - 1s 39us/step - loss: 1.4060 - accuracy: 0.3990 - val_loss: 1.3987 - val_accuracy: 0.4082
Epoch 13/80
20387/20387 [==============================] - 1s 40us/step - loss: 1.4013 - accuracy: 0.4003 - val_los

Epoch 23/80
20387/20387 [==============================] - 1s 38us/step - loss: 0.8499 - accuracy: 0.6537 - val_loss: 0.8271 - val_accuracy: 0.6752
Epoch 24/80
20387/20387 [==============================] - 1s 35us/step - loss: 0.8143 - accuracy: 0.6642 - val_loss: 0.8029 - val_accuracy: 0.6770
Epoch 25/80
20387/20387 [==============================] - 1s 36us/step - loss: 0.7831 - accuracy: 0.6735 - val_loss: 0.7812 - val_accuracy: 0.6690
Epoch 26/80
20387/20387 [==============================] - 1s 35us/step - loss: 0.7593 - accuracy: 0.6864 - val_loss: 0.8192 - val_accuracy: 0.6752
Epoch 27/80
20387/20387 [==============================] - 1s 34us/step - loss: 0.7388 - accuracy: 0.6943 - val_loss: 0.7393 - val_accuracy: 0.7004
Epoch 28/80
20387/20387 [==============================] - 1s 33us/step - loss: 0.7239 - accuracy: 0.6980 - val_loss: 0.7341 - val_accuracy: 0.7132
Epoch 29/80
20387/20387 [==============================] - 1s 36us/step - loss: 0.7073 - accuracy: 0.7032 - val_

20387/20387 [==============================] - 1s 34us/step - loss: 0.5721 - accuracy: 0.7500 - val_loss: 0.5905 - val_accuracy: 0.7524
Epoch 79/80
20387/20387 [==============================] - 1s 34us/step - loss: 0.5821 - accuracy: 0.7476 - val_loss: 0.5648 - val_accuracy: 0.7520
Epoch 80/80
20387/20387 [==============================] - 1s 36us/step - loss: 0.5773 - accuracy: 0.7506 - val_loss: 0.5617 - val_accuracy: 0.7582
[[5.0493799e-21 9.2349559e-02 1.7204155e-01 ... 2.1006361e-09
  9.0834931e-08 1.7361373e-11]
 [1.4823358e-12 5.3851698e-03 1.0641338e-10 ... 1.7036432e-04
  9.2008942e-01 2.7940553e-08]
 [1.5235250e-12 5.1862942e-03 1.0020483e-10 ... 1.6981087e-04
  9.2027634e-01 2.7657816e-08]
 ...
 [2.9811886e-01 0.0000000e+00 0.0000000e+00 ... 2.0095981e-18
  3.0718911e-10 4.3242874e-16]
 [4.3608427e-01 6.2611925e-32 0.0000000e+00 ... 2.2869129e-02
  6.3491891e-05 1.5779774e-01]
 [1.8991260e-11 1.2950096e-04 3.5180001e-11 ... 8.4406950e-02
  2.0313431e-03 1.9942336e-02]]
[12 

...

19471/19471 [==============================] - 1s 39us/step - loss: 1.3469 - accuracy: 0.4190 - val_loss: 1.3295 - val_accuracy: 0.4298
Epoch 34/80
19471/19471 [==============================] - 1s 39us/step - loss: 1.3424 - accuracy: 0.4221 - val_loss: 1.3126 - val_accuracy: 0.4330
Epoch 35/80
19471/19471 [==============================] - 1s 41us/step - loss: 1.3456 - accuracy: 0.4203 - val_loss: 1.3123 - val_accuracy: 0.4353
Epoch 36/80
19471/19471 [==============================] - 1s 40us/step - loss: 1.3454 - accuracy: 0.4213 - val_loss: 1.3269 - val_accuracy: 0.4339
Epoch 37/80
19471/19471 [==============================] - 1s 37us/step - loss: 1.3439 - accuracy: 0.4191 - val_loss: 1.3300 - val_accuracy: 0.4251
Epoch 38/80
19471/19471 [==============================] - 1s 39us/step - loss: 1.3413 - accuracy: 0.4194 - val_loss: 1.3360 - val_accuracy: 0.4164
Epoch 39/80
19471/19471 [==============================] - 1s 39us/step - loss: 1.3433 - accuracy: 0.4188 - val_loss: 1.3155

Train on 19426 samples, validate on 2159 samples
Epoch 1/80
19426/19426 [==============================] - 1s 55us/step - loss: 2.3583 - accuracy: 0.2582 - val_loss: 1.7301 - val_accuracy: 0.3506
Epoch 2/80
19426/19426 [==============================] - 1s 44us/step - loss: 1.5685 - accuracy: 0.3693 - val_loss: 1.4739 - val_accuracy: 0.3983
Epoch 3/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.4496 - accuracy: 0.3946 - val_loss: 1.4441 - val_accuracy: 0.3969
Epoch 4/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.4204 - accuracy: 0.3984 - val_loss: 1.3890 - val_accuracy: 0.3988
Epoch 5/80
19426/19426 [==============================] - 1s 41us/step - loss: 1.4022 - accuracy: 0.4032 - val_loss: 1.3652 - val_accuracy: 0.4141
Epoch 6/80
19426/19426 [==============================] - 1s 40us/step - loss: 1.3880 - accuracy: 0.4055 - val_loss: 1.3444 - val_accuracy: 0.4349
Epoch 7/80
19426/19426 [==============================] - 1s 41us/ste

19231/19231 [==============================] - 1s 41us/step - loss: 1.3503 - accuracy: 0.4256 - val_loss: 1.3487 - val_accuracy: 0.4385
Epoch 58/80
19231/19231 [==============================] - 1s 42us/step - loss: 1.3366 - accuracy: 0.4338 - val_loss: 1.3342 - val_accuracy: 0.4109
Epoch 59/80
19231/19231 [==============================] - 1s 38us/step - loss: 1.3250 - accuracy: 0.4467 - val_loss: 1.3197 - val_accuracy: 0.4759
Epoch 60/80
19231/19231 [==============================] - 1s 42us/step - loss: 1.3052 - accuracy: 0.4577 - val_loss: 1.3083 - val_accuracy: 0.4577
Epoch 61/80
19231/19231 [==============================] - 1s 39us/step - loss: 1.2802 - accuracy: 0.4732 - val_loss: 1.2699 - val_accuracy: 0.4726
Epoch 62/80
19231/19231 [==============================] - 1s 40us/step - loss: 1.2552 - accuracy: 0.4881 - val_loss: 1.2380 - val_accuracy: 0.4567
Epoch 63/80
19231/19231 [==============================] - 1s 38us/step - loss: 1.2284 - accuracy: 0.4987 - val_loss: 1.2154

Epoch 23/80
18674/18674 [==============================] - 1s 38us/step - loss: 1.3016 - accuracy: 0.4172 - val_loss: 1.2925 - val_accuracy: 0.4236
Epoch 24/80
18674/18674 [==============================] - 1s 40us/step - loss: 1.3002 - accuracy: 0.4186 - val_loss: 1.3140 - val_accuracy: 0.4135
Epoch 25/80
18674/18674 [==============================] - 1s 37us/step - loss: 1.2965 - accuracy: 0.4211 - val_loss: 1.2941 - val_accuracy: 0.4265
Epoch 26/80
18674/18674 [==============================] - 1s 38us/step - loss: 1.2994 - accuracy: 0.4231 - val_loss: 1.2940 - val_accuracy: 0.4270
Epoch 27/80
18674/18674 [==============================] - 1s 39us/step - loss: 1.2984 - accuracy: 0.4196 - val_loss: 1.2845 - val_accuracy: 0.4260
Epoch 28/80
18674/18674 [==============================] - 1s 38us/step - loss: 1.2947 - accuracy: 0.4232 - val_loss: 1.2959 - val_accuracy: 0.4039
Epoch 29/80
18674/18674 [==============================] - 1s 38us/step - loss: 1.2959 - accuracy: 0.4200 - val_

Epoch 40/80
19384/19384 [==============================] - 1s 41us/step - loss: 0.6741 - accuracy: 0.7166 - val_loss: 0.6851 - val_accuracy: 0.7177
Epoch 41/80
19384/19384 [==============================] - 1s 39us/step - loss: 0.6519 - accuracy: 0.7261 - val_loss: 0.6556 - val_accuracy: 0.7377
Epoch 42/80
19384/19384 [==============================] - 1s 40us/step - loss: 0.6520 - accuracy: 0.7229 - val_loss: 0.6812 - val_accuracy: 0.7140
Epoch 43/80
19384/19384 [==============================] - 1s 39us/step - loss: 0.6462 - accuracy: 0.7251 - val_loss: 0.6647 - val_accuracy: 0.7382
Epoch 44/80
19384/19384 [==============================] - 1s 39us/step - loss: 0.6390 - accuracy: 0.7286 - val_loss: 0.6531 - val_accuracy: 0.7317
Epoch 45/80
19384/19384 [==============================] - 1s 39us/step - loss: 0.6310 - accuracy: 0.7294 - val_loss: 0.6622 - val_accuracy: 0.7340
Epoch 46/80
19384/19384 [==============================] - 1s 39us/step - loss: 0.6264 - accuracy: 0.7333 - val_

In [12]:
# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
            
    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)


n= 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3, 6)              12        
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 6)              0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 16)             112       
_________________________________________________________________
average_pooling1d_2 (Average (None, 1, 16)             0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 120)            2040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 84)          

Epoch 7/80
19373/19373 [==============================] - 1s 37us/step - loss: 1.3938 - accuracy: 0.3964 - val_loss: 1.3955 - val_accuracy: 0.3985
Epoch 8/80
19373/19373 [==============================] - 1s 40us/step - loss: 1.3882 - accuracy: 0.4012 - val_loss: 1.3880 - val_accuracy: 0.3985
Epoch 9/80
19373/19373 [==============================] - 1s 40us/step - loss: 1.3837 - accuracy: 0.3998 - val_loss: 1.3763 - val_accuracy: 0.4129
Epoch 10/80
19373/19373 [==============================] - 1s 41us/step - loss: 1.3784 - accuracy: 0.4074 - val_loss: 1.3737 - val_accuracy: 0.4069
Epoch 11/80
19373/19373 [==============================] - 1s 40us/step - loss: 1.3713 - accuracy: 0.4032 - val_loss: 1.3601 - val_accuracy: 0.4148
Epoch 12/80
19373/19373 [==============================] - 1s 39us/step - loss: 1.3696 - accuracy: 0.4060 - val_loss: 1.3613 - val_accuracy: 0.4143
Epoch 13/80
19373/19373 [==============================] - 1s 40us/step - loss: 1.3622 - accuracy: 0.4043 - val_los

Epoch 30/80
19684/19684 [==============================] - 1s 39us/step - loss: 1.3472 - accuracy: 0.4125 - val_loss: 1.3713 - val_accuracy: 0.3985
Epoch 31/80
19684/19684 [==============================] - 1s 37us/step - loss: 1.3464 - accuracy: 0.4099 - val_loss: 1.3823 - val_accuracy: 0.3912
Epoch 32/80
19684/19684 [==============================] - 1s 37us/step - loss: 1.3402 - accuracy: 0.4154 - val_loss: 1.3752 - val_accuracy: 0.3958
Epoch 33/80
19684/19684 [==============================] - 1s 37us/step - loss: 1.3442 - accuracy: 0.4129 - val_loss: 1.3668 - val_accuracy: 0.4068
Epoch 34/80
19684/19684 [==============================] - 1s 39us/step - loss: 1.3427 - accuracy: 0.4106 - val_loss: 1.3660 - val_accuracy: 0.3935
Epoch 35/80
19684/19684 [==============================] - 1s 40us/step - loss: 1.3422 - accuracy: 0.4120 - val_loss: 1.3828 - val_accuracy: 0.3894
Epoch 36/80
19684/19684 [==============================] - 1s 39us/step - loss: 1.3395 - accuracy: 0.4119 - val_

19354/19354 [==============================] - 1s 37us/step - loss: 0.7756 - accuracy: 0.6964 - val_loss: 0.7336 - val_accuracy: 0.7141
Epoch 77/80
19354/19354 [==============================] - 1s 39us/step - loss: 0.7528 - accuracy: 0.7029 - val_loss: 0.7295 - val_accuracy: 0.7215
Epoch 78/80
19354/19354 [==============================] - 1s 42us/step - loss: 0.7279 - accuracy: 0.7093 - val_loss: 0.6950 - val_accuracy: 0.7350
Epoch 79/80
19354/19354 [==============================] - 1s 41us/step - loss: 0.7182 - accuracy: 0.7087 - val_loss: 0.6971 - val_accuracy: 0.7104
Epoch 80/80
19354/19354 [==============================] - 1s 39us/step - loss: 0.7053 - accuracy: 0.7138 - val_loss: 0.6645 - val_accuracy: 0.7364
[[1.0440017e-22 7.6406419e-01 5.6053275e-37 ... 2.7090305e-10
  1.4319590e-06 6.0942230e-11]
 [1.2619887e-13 6.4819950e-01 3.0843669e-28 ... 2.2517145e-03
  2.5052765e-01 4.9747705e-06]
 [1.3204214e-13 6.4282453e-01 3.2223451e-28 ... 2.3266578e-03
  2.5644547e-01 5.087300

Train on 19196 samples, validate on 2133 samples
Epoch 1/80
19196/19196 [==============================] - 1s 49us/step - loss: 2.4130 - accuracy: 0.2179 - val_loss: 1.7991 - val_accuracy: 0.2982
Epoch 2/80
19196/19196 [==============================] - 1s 39us/step - loss: 1.6012 - accuracy: 0.3524 - val_loss: 1.5026 - val_accuracy: 0.3840
Epoch 3/80
19196/19196 [==============================] - 1s 37us/step - loss: 1.4805 - accuracy: 0.3834 - val_loss: 1.4534 - val_accuracy: 0.4060
Epoch 4/80
19196/19196 [==============================] - 1s 36us/step - loss: 1.4464 - accuracy: 0.4004 - val_loss: 1.4363 - val_accuracy: 0.3910
Epoch 5/80
19196/19196 [==============================] - 1s 37us/step - loss: 1.4293 - accuracy: 0.3986 - val_loss: 1.4382 - val_accuracy: 0.3891
Epoch 6/80
19196/19196 [==============================] - 1s 39us/step - loss: 1.4247 - accuracy: 0.3997 - val_loss: 1.4154 - val_accuracy: 0.3976
Epoch 7/80
19196/19196 [==============================] - 1s 37us/ste

...

19426/19426 [==============================] - 1s 39us/step - loss: 1.3341 - accuracy: 0.4229 - val_loss: 1.3151 - val_accuracy: 0.4363
Epoch 24/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.3351 - accuracy: 0.4241 - val_loss: 1.3115 - val_accuracy: 0.4224
Epoch 25/80
19426/19426 [==============================] - 1s 40us/step - loss: 1.3303 - accuracy: 0.4264 - val_loss: 1.3107 - val_accuracy: 0.4368
Epoch 26/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.3270 - accuracy: 0.4245 - val_loss: 1.3049 - val_accuracy: 0.4275
Epoch 27/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.3250 - accuracy: 0.4260 - val_loss: 1.3117 - val_accuracy: 0.4382
Epoch 28/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.3272 - accuracy: 0.4248 - val_loss: 1.3068 - val_accuracy: 0.4396
Epoch 29/80
19426/19426 [==============================] - 1s 39us/step - loss: 1.3268 - accuracy: 0.4245 - val_loss: 1.2981

19231/19231 [==============================] - 1s 40us/step - loss: 1.2967 - accuracy: 0.4692 - val_loss: 1.2715 - val_accuracy: 0.4848
Epoch 40/80
19231/19231 [==============================] - 1s 42us/step - loss: 1.2522 - accuracy: 0.4987 - val_loss: 1.2091 - val_accuracy: 0.5316
Epoch 41/80
19231/19231 [==============================] - 1s 40us/step - loss: 1.1914 - accuracy: 0.5338 - val_loss: 1.1413 - val_accuracy: 0.5690
Epoch 42/80
19231/19231 [==============================] - 1s 40us/step - loss: 1.1148 - accuracy: 0.5767 - val_loss: 1.0653 - val_accuracy: 0.6149
Epoch 43/80
19231/19231 [==============================] - 1s 41us/step - loss: 1.0196 - accuracy: 0.6126 - val_loss: 0.9751 - val_accuracy: 0.6364
Epoch 44/80
19231/19231 [==============================] - 1s 40us/step - loss: 0.9402 - accuracy: 0.6387 - val_loss: 0.8992 - val_accuracy: 0.6476
Epoch 45/80
19231/19231 [==============================] - 1s 43us/step - loss: 0.8796 - accuracy: 0.6596 - val_loss: 0.8537

Train on 19178 samples, validate on 2131 samples
Epoch 1/80
19178/19178 [==============================] - 1s 51us/step - loss: 2.3282 - accuracy: 0.2286 - val_loss: 1.7175 - val_accuracy: 0.3651
Epoch 2/80
19178/19178 [==============================] - 1s 38us/step - loss: 1.5545 - accuracy: 0.3718 - val_loss: 1.4505 - val_accuracy: 0.4045
Epoch 3/80
19178/19178 [==============================] - 1s 37us/step - loss: 1.4307 - accuracy: 0.3919 - val_loss: 1.4082 - val_accuracy: 0.3801
Epoch 4/80
19178/19178 [==============================] - 1s 38us/step - loss: 1.4033 - accuracy: 0.3986 - val_loss: 1.3774 - val_accuracy: 0.4205
Epoch 5/80
19178/19178 [==============================] - 1s 39us/step - loss: 1.3833 - accuracy: 0.3995 - val_loss: 1.3548 - val_accuracy: 0.4083
Epoch 6/80
19178/19178 [==============================] - 1s 40us/step - loss: 1.3685 - accuracy: 0.4118 - val_loss: 1.3617 - val_accuracy: 0.4078
Epoch 7/80
19178/19178 [==============================] - 1s 40us/ste

18674/18674 [==============================] - 1s 40us/step - loss: 1.3129 - accuracy: 0.4139 - val_loss: 1.2793 - val_accuracy: 0.4270
Epoch 18/80
18674/18674 [==============================] - 1s 40us/step - loss: 1.3084 - accuracy: 0.4196 - val_loss: 1.2852 - val_accuracy: 0.4270
Epoch 19/80
18674/18674 [==============================] - 1s 38us/step - loss: 1.3117 - accuracy: 0.4217 - val_loss: 1.2866 - val_accuracy: 0.4101
Epoch 20/80
18674/18674 [==============================] - 1s 42us/step - loss: 1.3032 - accuracy: 0.4159 - val_loss: 1.2974 - val_accuracy: 0.4087
Epoch 21/80
18674/18674 [==============================] - 1s 44us/step - loss: 1.3056 - accuracy: 0.4150 - val_loss: 1.3265 - val_accuracy: 0.4048
Epoch 22/80
18674/18674 [==============================] - 1s 39us/step - loss: 1.3067 - accuracy: 0.4136 - val_loss: 1.2827 - val_accuracy: 0.4202
Epoch 23/80
18674/18674 [==============================] - 1s 37us/step - loss: 1.2990 - accuracy: 0.4183 - val_loss: 1.2879

19384/19384 [==============================] - 1s 39us/step - loss: 1.3008 - accuracy: 0.4171 - val_loss: 1.2873 - val_accuracy: 0.4401
Epoch 34/80
19384/19384 [==============================] - 1s 41us/step - loss: 1.2987 - accuracy: 0.4228 - val_loss: 1.2964 - val_accuracy: 0.4099
Epoch 35/80
19384/19384 [==============================] - 1s 36us/step - loss: 1.2951 - accuracy: 0.4207 - val_loss: 1.2928 - val_accuracy: 0.4359
Epoch 36/80
19384/19384 [==============================] - 1s 37us/step - loss: 1.2952 - accuracy: 0.4212 - val_loss: 1.2790 - val_accuracy: 0.4327
Epoch 37/80
19384/19384 [==============================] - 1s 39us/step - loss: 1.2942 - accuracy: 0.4211 - val_loss: 1.2923 - val_accuracy: 0.4136
Epoch 38/80
19384/19384 [==============================] - 1s 35us/step - loss: 1.2983 - accuracy: 0.4232 - val_loss: 1.2965 - val_accuracy: 0.4262
Epoch 39/80
19384/19384 [==============================] - 1s 37us/step - loss: 1.2933 - accuracy: 0.4245 - val_loss: 1.2928

In [13]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

(153, 43217) (153, 43217, 17) (153, 17)
normal size =  (153, 43217, 17)
(18, 43217) (18, 43217, 17) (18, 17)


In [14]:
print("done")

done
